In [2]:
import parametric as param1
import parametric2 as param2
import numpy as np
import gc

In [3]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
fastclime = importr('fastclime')
grdevices = importr('grDevices')
base = importr('base')
stats = importr('stats')

In [4]:
def is_Hermitian(m):
    """
    Checks if a given matrix is Hermitian 
    (symmetric)

    Parameters:
    -----------------------------------------
    m   :  A 2-D matrix

    Returns:
    -----------------------------------------
    logical
    """
    import numpy as np
    
    #Set missing to zero prior to checking symmetry
    m[np.isnan(m)] = 0.

    try:
        return np.allclose(np.transpose(m,(1,0)), m)
    except:
        return False
    
def symmetrize(m,rule="min"):
    """
    Symmetrizes a given square matrix based on a rule

    Parameters:
    -----------------------------------------
    m   :  A square matrix

    rule:  criterion for symmetrizing m
           -"min" computes the minimum of m(i,j) and m(j,i)
           -"max" computes the maximum of m(i,j) and m(j,i)
           where i and j are row and column indices

    Returns:
    -----------------------------------------
    A symmetric square matrix
    """
    import numpy as np
    
    if (m.shape[0] != m.shape[1]):
        raise ValueError("Input matrix must be square.")
        
    if (rule == "min"):
        min_mat =  np.fmin(np.triu(m),np.tril(m).T)
        return np.triu(min_mat,1) + min_mat.T

    elif (rule == "max"):
        max_mat =  np.fmax(np.triu(m),np.tril(m).T)
        return np.triu(max_mat,1) + max_mat.T
    
    else:
        raise ValueError("Specify rule as min or max.")

def fastclime1(x,lambda_min=0.1,nlambda=50):
 
    cov_input = 1
    SigmaInput = x.copy()
    
    #Check if matrix is symmetric
    if not is_Hermitian(SigmaInput):
        SigmaInput = np.corrcoef(SigmaInput.T)
        cov_input = 0
    
    #Run parametric simplex linear solver
    Sigmahat, mu, maxnlambda, iicov = param1.mainfunc(SigmaInput,lambda_min,nlambda)
      
    #Process output
    maxnlambda+=1
    
    #Reshape the array in Fortran order
    #and then slice the array to extract only the top maxnlambda rows
    lambdamtx = mu.T.reshape(nlambda, -1, order='F')[:maxnlambda,:]
    mu = None
    
    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
   
    #Symmetrize output precision matrices 
    for i in range(maxnlambda):
        #icovlist[:,:,i] = symmetrize(iicov[:,i].reshape((d,d)).T,"min")
        icovlist[:,:,i] = iicov[:,i].reshape((d,d)).T
    
    gc.collect()
    
    return x, cov_input, Sigmahat, maxnlambda, lambdamtx, icovlist
    
    
def fastclime2(x,SigmaInput_xy,lambda_min=0.1,nlambda=50):

    cov_input = 1
    SigmaInput_xx = x.copy()
    
    #Check if matrix is symmetric
    if not is_Hermitian(SigmaInput_xx):
        SigmaInput_xx = np.corrcoef(SigmaInput_xx.T)
        cov_input = 0
    
    #Check that the shape of SigmaInput_xx and SigmaInput_xy are the same
    if (SigmaInput_xx.shape != SigmaInput_xy.shape):
        raise ValueError("Dimensions for input covariance matrices must match.")
    
    #Run parametric simplex linear solver
    Sigmahat, mu, maxnlambda, iicov = param2.mainfunc(SigmaInput_xx,SigmaInput_xy,
                                                     lambda_min,nlambda)
      
    #Process output
    maxnlambda+=1
    
    #Reshape the array in Fortran order
    #and then slice the array to extract only the top maxnlambda rows
    lambdamtx = mu.T.reshape(nlambda, -1, order='F')[:maxnlambda,:]
    mu = None
    
    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
   
    #Symmetrize output precision matrices 
    for i in range(maxnlambda):
        #icovlist[:,:,i] = symmetrize(iicov[:,i].reshape((d,d)).T,"min")
         icovlist[:,:,i] = iicov[:,i].reshape((d,d)).T
    
    gc.collect()
    
    return x, cov_input, Sigmahat, maxnlambda, lambdamtx, icovlist
    

In [5]:
L = fastclime.fastclime_generator(n = 10, d = 5)
pydat = np.array(L.rx2('data'))

Generating data from the multivariate normal distribution with the random graph structure....done.


In [6]:
Pyout = fastclime1(pydat)

In [ ]:
from numpy import matlib
Imat = np.matlib.eye(5)
Pyout2 = fastclime2(pydat,Imat)


In [ ]:
Rout = fastclime.fastclime(L.rx2('data'),0.1)